In [14]:
import os
import re
import struct
import io
import tempfile
import random
from typing import Tuple
from wand.image import Image as WandImage

from PIL import Image

ACFA_THUMBNAIL_HEADER = bytes([
    0x10, 0x00, 0x00, 0x80, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x40, 0x00
])



def parse_part_mapping(file_path):
    part_mapping = {}
    current_category = None

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            # Detect category header like 'Head (0):'
            if line.endswith('):'):
                category_name, category_id = line[:-2].rsplit('(', 1)
                current_category = category_name.strip()
                part_mapping[current_category] = {}
                continue

            if current_category is None:
                continue  # Skip any lines before first category

            # Split the line into part_id and part_name
            if ' ' in line:
                part_id, part_name = line.split(' ', 1)
                part_mapping[current_category][part_id.strip()
                                               ] = part_name.strip()

    return part_mapping


part_mapping = parse_part_mapping("ACFA_PS3_US_PARTID_TO_PARTNAME.txt")
print(part_mapping.keys())

BLOCK_SIZE = 24280
NAME_SIZE = 96  # 48 wchar_t = 96 bytes in UTF-16


def load_file(path):
    with open(path, "rb") as f:
        return f.read()


def save_file(path, data):
    with open(path, "wb") as f:
        f.write(data)

def linear_utf16_clean_name_reader(data, start_offset, max_bytes=96):
    raw_field = data[start_offset:start_offset + max_bytes]
    try:
        decoded = raw_field.decode('utf-16-le', errors='ignore').strip('\x00')
        match = re.match(r'^[A-Za-z0-9 ]+', decoded)
        if match:
            return match.group(0).strip()
        return "<Invalid UTF-16 Encoding>"
    except UnicodeDecodeError:
        return "<Invalid UTF-16 Encoding>"


def read_timestamp(data, offset):
    timestamp_bytes = data[offset:offset + 8]
    return struct.unpack(">Q", timestamp_bytes)[0]


def extract_active_schematic_blocks(file_path):
    """
    Extracts all schematic blocks from the given file.
    Returns a list of blocks.
    """
    data = load_file(file_path)
    schematic_count = data[5]
    blocks = []
    first_marker_offset = 0x148

    for slot_index in range(schematic_count):
        block_start = first_marker_offset + (slot_index * BLOCK_SIZE)
        block = data[block_start:block_start + BLOCK_SIZE]
        blocks.append(block)
    return blocks


def display_schematic_info(block):
    """
    Displays the schematic information from a block.
    Returns a dictionary with the schematic information.
    """
    schematic_name = linear_utf16_clean_name_reader(block, 1, NAME_SIZE)
    designer_name = linear_utf16_clean_name_reader(
        block, 1 + NAME_SIZE, NAME_SIZE)
    timestamp = read_timestamp(block, 192)

    protect_category_byte = block[200]
    protect = (protect_category_byte & 0b10000000) >> 7
    category = (protect_category_byte & 0b01111111) + 1

    parts = extract_parts(block, part_mapping)
    tuning = extract_tuning(block)

    schematic_info = {
        "name": schematic_name,
        "designer": designer_name,
        "category": category,
        "timestamp": timestamp,
        "parts": parts,
        "tuning": tuning
    }

    # Optional print for debugging
    # print(f"  Name: {schematic_info['name']}")
    # print(f"  Designer: {schematic_info['designer']}")
    # print(f"  Protect: {schematic_info['protect']}")
    # print(f"  User Slot: {schematic_info['category']}")
    # print(f"  Raw Byte: 0x{schematic_info['raw_byte']:02x}")
    # print(f"  Timestamp: {schematic_info['timestamp']}")
    # print("  Parts:", schematic_info['parts'])
    # print("  Tuning:", schematic_info['tuning'])
    # print()

    return schematic_info


def extract_parts(block, part_name_lookup):
    LOCAL_PARTS_OFFSET = 0xD8  # 0x220 - 0x148
    PART_ENTRY_SIZE = 2

    # Define lookup keys and display labels separately
    lookup_keys = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Arm Unit', 'Arm Unit', 'Back Unit', 'Back Unit', 'Shoulder Unit'
    ]

    display_labels = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
        'Left Back Unit', 'Shoulder Unit'
    ]

    parts_info = []
    for i, (lookup_key, display_label) in enumerate(zip(lookup_keys, display_labels)):
        offset = LOCAL_PARTS_OFFSET + i * PART_ENTRY_SIZE
        part_id_bytes = block[offset:offset + PART_ENTRY_SIZE]

        if len(part_id_bytes) != 2:
            part_id_str = "<Invalid>"
            part_name = "<Invalid>"
        else:
            part_id_num = int.from_bytes(part_id_bytes, byteorder='big')
            part_id_str = f"{part_id_num:04d}"
            part_name = part_name_lookup.get(lookup_key, {}).get(
                part_id_str, f"Unknown ID {part_id_str}")

        parts_info.append({
            "category": display_label,
            "part_id": part_id_str,
            "part_name": part_name
        })

    return parts_info


def extract_tuning(block):
    LOCAL_TUNING_OFFSET = 0x126  # 0x26E - 0x148
    TUNING_SIZE = 32  # 0x20 bytes

    tuning_labels = [
        'en_output',
        'en_capacity',
        'kp_output',
        'load',
        'en_weapon_skill',
        'maneuverability',
        'firing_stability',
        'aim_precision',
        'lock_speed',
        'missile_lock_speed',
        'radar_refresh_rate',
        'ecm_resistance',
        'rectification_head',
        'rectification_core',
        'rectification_arm',
        'rectification_leg',
        'horizontal_thrust_main',
        'vertical_thrust',
        'horizontal_thrust_side',
        'horizontal_thrust_back',
        'quick_boost_main',
        'quick_boost_side',
        'quick_boost_back',
        'quick_boost_overed',
        'turning_ability',
        'stability_head',
        'stability_core',
        'stability_legs',
    ]

    tuning_values = {}
    for i, label in enumerate(tuning_labels):
        value = block[LOCAL_TUNING_OFFSET + i]
        tuning_values[label] = value  # Value should be in range 0-50

    return tuning_values

def extract_color_data(schematic_block: bytes) -> Tuple[bytearray, bytearray, bytearray]:
    """
    Extracts color, pattern, and eye color data from a schematic block.

    The function calculates the local offsets for each data type based on their
    known absolute positions in the save file and returns them as mutable
    bytearrays.

    Args:
        schematic_block: A bytes object representing a single schematic block.

    Returns:
        A tuple containing three bytearrays:
        1. The main color data block (0x330 bytes).
        2. The pattern data block (0x24 bytes).
        3. The eye color data (0x4 bytes).
    """
    # Absolute address of the first schematic block in DESDOC.DAT
    SCHEMATIC_START_ABS = 0x148

    # --- Calculate Local Offsets ---
    # Colors: 0x290 (absolute) - 0x148 (schematic start) = 0x148 (local)
    COLORS_LOCAL_OFFSET = 0x148
    COLORS_SIZE = 0x330

    # Patterns: 0x5C0 (absolute) - 0x148 (schematic start) = 0x478 (local)
    PATTERNS_LOCAL_OFFSET = 0x478
    PATTERNS_SIZE = 0x24

    # Eye Color: 0x5E4 (absolute) - 0x148 (schematic start) = 0x49C (local)
    EYE_COLOR_LOCAL_OFFSET = 0x49C
    EYE_COLOR_SIZE = 0x4

    # --- Extract Data Blocks ---
    colors_data = schematic_block[COLORS_LOCAL_OFFSET:COLORS_LOCAL_OFFSET + COLORS_SIZE]
    patterns_data = schematic_block[PATTERNS_LOCAL_OFFSET:PATTERNS_LOCAL_OFFSET + PATTERNS_SIZE]
    eye_color_data = schematic_block[EYE_COLOR_LOCAL_OFFSET:EYE_COLOR_LOCAL_OFFSET + EYE_COLOR_SIZE]

    # Return as mutable bytearrays
    return bytearray(colors_data), bytearray(patterns_data), bytearray(eye_color_data)


def replace_color_data(
    schematic_block: bytes,
    new_colors: bytes | None = None,
    new_patterns: bytes | None = None,
    new_eye_color: bytes | None = None
) -> bytes:
    """
    Replaces specified color, pattern, or eye color data in a schematic block.

    If a replacement is not provided for a specific data block (i.e., the
    argument is None), the original data from the schematic_block is kept.

    Args:
        schematic_block: The original schematic block bytes.
        new_colors: Optional new color data (must be 0x330 bytes if provided).
        new_patterns: Optional new pattern data (must be 0x24 bytes if provided).
        new_eye_color: Optional new eye color data (must be 0x4 bytes if provided).

    Returns:
        A new schematic block bytes object with the specified data replaced.
    """
    # Define constants for offsets and sizes
    COLORS_LOCAL_OFFSET = 0x148
    COLORS_SIZE = 0x330
    PATTERNS_LOCAL_OFFSET = 0x478
    PATTERNS_SIZE = 0x24
    EYE_COLOR_LOCAL_OFFSET = 0x49C
    EYE_COLOR_SIZE = 0x4

    mutable_block = bytearray(schematic_block)

    # --- Conditionally Replace Data ---
    if new_colors is not None:
        if len(new_colors) != COLORS_SIZE:
            raise ValueError(
                f"Invalid colors data size. Expected {COLORS_SIZE}, got {len(new_colors)}.")
        mutable_block[COLORS_LOCAL_OFFSET:COLORS_LOCAL_OFFSET +
                      COLORS_SIZE] = new_colors

    if new_patterns is not None:
        if len(new_patterns) != PATTERNS_SIZE:
            raise ValueError(
                f"Invalid patterns data size. Expected {PATTERNS_SIZE}, got {len(new_patterns)}.")
        mutable_block[PATTERNS_LOCAL_OFFSET:PATTERNS_LOCAL_OFFSET +
                      PATTERNS_SIZE] = new_patterns

    if new_eye_color is not None:
        if len(new_eye_color) != EYE_COLOR_SIZE:
            raise ValueError(
                f"Invalid eye color data size. Expected {EYE_COLOR_SIZE}, got {len(new_eye_color)}.")
        mutable_block[EYE_COLOR_LOCAL_OFFSET:EYE_COLOR_LOCAL_OFFSET +
                      EYE_COLOR_SIZE] = new_eye_color

    return bytes(mutable_block)


def randomize_colors(colors_data: bytes) -> bytes:
    """
    Randomizes the RGB channels for all colors in a color data block.

    The alpha channel of each color is preserved as it is unused in-game.

    Args:
        colors_data: The color data block (e.g., 0x330 bytes).

    Returns:
        A new color data block with randomized RGB values.
    """
    if len(colors_data) % 4 != 0:
        raise ValueError(
            "Invalid colors data length. Length must be divisible by 4.")

    mutable_colors = bytearray(colors_data)

    # Iterate through each color (4 bytes at a time)
    for i in range(0, len(mutable_colors), 4):
        # Randomize R, G, B channels
        mutable_colors[i] = random.randint(0, 255)   # R
        mutable_colors[i+1] = random.randint(0, 255)  # G
        mutable_colors[i+2] = random.randint(0, 255)  # B
        # The 4th byte (alpha) at i+3 is intentionally left unchanged.

    return bytes(mutable_colors)


def extract_decal_data(schematic_block: bytes) -> bytearray:
    """
    Extracts decal data from a schematic block.

    The function calculates the local offset for the decal data based on its
    known absolute position in the save file and returns it as a mutable
    bytearray.

    Args:
        schematic_block: A bytes object representing a single schematic block.

    Returns:
        A bytearray containing the decal data (0x19A0 bytes).
    """
    # Absolute address of the first schematic block in DESDOC.DAT
    SCHEMATIC_START_ABS = 0x148

    # Decals: 0x5E8 (absolute) - 0x148 (schematic start) = 0x4A0 (local)
    DECAL_DATA_LOCAL_OFFSET = 0x4A0
    DECAL_DATA_SIZE = 0x19A0

    # --- Extract Data Block ---
    decal_data = schematic_block[DECAL_DATA_LOCAL_OFFSET:
                                 DECAL_DATA_LOCAL_OFFSET + DECAL_DATA_SIZE]

    # Return as a mutable bytearray
    return bytearray(decal_data)


def replace_decal_data(
    schematic_block: bytes,
    new_decal_data: bytes
) -> bytes:
    """
    Replaces the decal data in a schematic block.

    Args:
        schematic_block: The original schematic block bytes.
        new_decal_data: New decal data (must be 0x19A0 bytes).

    Returns:
        A new schematic block bytes object with the decal data replaced.
    """
    # Define constants for offset and size
    DECAL_DATA_LOCAL_OFFSET = 0x4A0
    DECAL_DATA_SIZE = 0x19A0

    if len(new_decal_data) != DECAL_DATA_SIZE:
        raise ValueError(
            f"Invalid decal data size. Expected {DECAL_DATA_SIZE}, got {len(new_decal_data)}.")

    mutable_block = bytearray(schematic_block)
    mutable_block[DECAL_DATA_LOCAL_OFFSET:DECAL_DATA_LOCAL_OFFSET +
                  DECAL_DATA_SIZE] = new_decal_data

    return bytes(mutable_block)


def parse_emblem_data(data: bytes):
    """
    Parses a 132-byte emblem data block into a structured dictionary.

    Args:
        data: A bytes object of length 132 (0x84).

    Returns:
        A dictionary containing the emblem's type and a list of 16 layers,
        with each layer's properties parsed into a human-readable format.
        Returns None if the data length is incorrect.
    """
    if len(data) != 132:
        raise ValueError(
            f"Invalid data length. Expected 132 bytes, got {len(data)}.")

    emblem_info = {
        'type': data[0],
        'unknown_header': data[1:4].hex(),
        'layers': []
    }

    layers_data = data[4:]

    for i in range(16):
        offset = i * 8
        layer_bytes = layers_data[offset:offset + 8]

        if len(layer_bytes) < 8:
            # Avoids index out of range if data is malformed
            continue

        flags = layer_bytes[7]

        layer_info = {
            'layer_index': i,
            'angle': layer_bytes[0],
            'image_id': layer_bytes[1],
            'color': layer_bytes[2],
            'width': layer_bytes[3],
            'height': layer_bytes[4],
            'x_position': layer_bytes[5],
            'y_position': layer_bytes[6],
            'flags': {
                'raw_byte': f"0x{flags:02x}",
                'negative_angle': bool((flags >> 4) & 1),
                'negative_x': bool((flags >> 6) & 1),
                'negative_y': bool((flags >> 7) & 1),
            }
        }
        emblem_info['layers'].append(layer_info)

    return emblem_info


def parse_paint_dat(file_path: str) -> list[bytes]:
    """
    Parses the PAINT.DAT file to extract existing emblem data blocks.

    Emblems start at offset 0x214. Each emblem is 132 bytes (0x84) long.
    There are 64 possible emblem slots. Parsing stops when an emblem's
    first byte is 0x00, indicating a non-existent emblem.

    Args:
        file_path: The absolute path to the PAINT.DAT file.

    Returns:
        A list of bytes objects, where each bytes object is a 132-byte
        emblem data block.
    """
    EMBLEM_START_OFFSET = 0x214
    EMBLEM_SIZE = 132  # 0x84 bytes
    NUM_EMBLEM_SLOTS = 64

    try:
        with open(file_path, "rb") as f:
            paint_data = f.read()
    except FileNotFoundError:
        raise FileNotFoundError(f"PAINT.DAT not found at {file_path}")

    emblems = []
    for i in range(NUM_EMBLEM_SLOTS):
        current_emblem_offset = EMBLEM_START_OFFSET + (i * EMBLEM_SIZE)

        # Ensure there's enough data to read at least the first byte of an emblem
        if current_emblem_offset >= len(paint_data):
            break

        # Check if the emblem exists (first byte is not 0x00)
        if paint_data[current_emblem_offset] == 0x00:
            break  # Stop parsing if a non-existent emblem is found

        # Extract the full emblem data block
        emblem_data = paint_data[current_emblem_offset:
                                 current_emblem_offset + EMBLEM_SIZE]

        # Ensure the extracted block is the correct size
        if len(emblem_data) == EMBLEM_SIZE:
            emblems.append(emblem_data)
        else:
            # This case should ideally not happen if the file is well-formed
            # and the previous length check passed, but good for robustness.
            print(
                f"Warning: Incomplete emblem data found at slot {i}. Skipping.")
            break  # Stop if an incomplete emblem is found

    return emblems


def generate_random_emblem(num_layers: int | None = None) -> bytes:
    """
    Generates a random 132-byte emblem data block adhering to game limitations.

    Args:
        num_layers: Optional. The number of layers to generate (1-16). If None,
                    a random number of layers will be generated.

    Returns:
        A bytes object representing a randomly generated emblem.
    """
    EMBLEM_SIZE = 132  # 0x84 bytes
    emblem_data = bytearray(EMBLEM_SIZE)

    # Header (4 bytes)
    emblem_data[0] = 0x02  # Type: custom emblem
    emblem_data[1:4] = bytes([0x00, 0x00, 0x00])  # Unknown, always 0

    # Valid image IDs from documentation
    valid_image_ids = []
    for r in [(0, 20), (29, 60), (69, 88), (97, 112), (121, 144), (149, 164), (173, 188), (205, 252)]:
        valid_image_ids.extend(range(r[0], r[1] + 1))

    if num_layers is None:
        actual_num_layers = random.randint(1, 16)
    elif 1 <= num_layers <= 16:
        actual_num_layers = num_layers
    else:
        raise ValueError("num_layers must be between 1 and 16, or None.")

    # Layers (8 bytes each)
    for i in range(actual_num_layers):
        layer_offset = 4 + (i * 8)

        # byte 0 : angle (0-180 decimal)
        emblem_data[layer_offset] = random.randint(0, 180)

        # byte 1 : image id
        emblem_data[layer_offset + 1] = random.choice(valid_image_ids)

        # byte 2 : color (0-7 decimal)
        emblem_data[layer_offset + 2] = random.randint(0, 7)

        # byte 3 : width (1-127 decimal)
        emblem_data[layer_offset + 3] = random.randint(1, 127)

        # byte 4 : height (1-127 decimal)
        emblem_data[layer_offset + 4] = random.randint(1, 127)

        # byte 5 : x position (0-255 decimal)
        emblem_data[layer_offset + 5] = random.randint(0, 255)

        # byte 6 : y position (0-255 decimal)
        emblem_data[layer_offset + 6] = random.randint(0, 255)

        # byte 7 : flags
        flags = 0
        # bit 0 - 3 : unknown, always 0
        # bit 4 : negative angle (0 or 1)
        flags |= (random.randint(0, 1) << 4)
        # bit 5 : unknown, always 0
        # bit 6 : negative x (0 or 1)
        flags |= (random.randint(0, 1) << 6)
        # bit 7 : negative y (0 or 1)
        flags |= (random.randint(0, 1) << 7)
        emblem_data[layer_offset + 7] = flags

    # Fill remaining layers with zeros if actual_num_layers < 16
    for i in range(actual_num_layers, 16):
        layer_offset = 4 + (i * 8)
        emblem_data[layer_offset:layer_offset + 8] = bytes([0] * 8)

    return bytes(emblem_data)



def append_emblem_to_paint_dat(file_path: str, new_emblem_data: bytes):
    """
    Appends a new emblem to the PAINT.DAT file by replacing the first empty slot.

    Args:
        file_path: The absolute path to the PAINT.DAT file.
        new_emblem_data: A bytes object representing the new emblem (must be 132 bytes).

    Raises:
        ValueError: If new_emblem_data is not 132 bytes long.
        FileNotFoundError: If PAINT.DAT is not found.
        RuntimeError: If no empty emblem slot is found in PAINT.DAT.
    """
    EMBLEM_START_OFFSET = 0x214
    EMBLEM_SIZE = 132  # 0x84 bytes
    NUM_EMBLEM_SLOTS = 64

    if len(new_emblem_data) != EMBLEM_SIZE:
        raise ValueError(
            f"New emblem data must be {EMBLEM_SIZE} bytes long, got {len(new_emblem_data)}.")

    try:
        with open(file_path, "rb") as f:
            paint_data = bytearray(f.read())
    except FileNotFoundError:
        raise FileNotFoundError(f"PAINT.DAT not found at {file_path}")

    found_empty_slot = False
    for i in range(NUM_EMBLEM_SLOTS):
        current_emblem_offset = EMBLEM_START_OFFSET + (i * EMBLEM_SIZE)

        # Ensure there's enough space in the file for this slot
        if current_emblem_offset + EMBLEM_SIZE > len(paint_data):
            # If we run out of file before finding an empty slot, it's an issue
            raise RuntimeError(
                "PAINT.DAT file is too short or corrupted, no space for new emblem.")

        # Check if the emblem slot is empty (first byte is 0x00)
        if paint_data[current_emblem_offset] == 0x00:
            # Replace the empty slot with the new emblem data
            paint_data[current_emblem_offset:current_emblem_offset +
                       EMBLEM_SIZE] = new_emblem_data
            found_empty_slot = True
            break

    if not found_empty_slot:
        raise RuntimeError(
            "No empty emblem slots found in PAINT.DAT. File is full.")

    # Save the modified PAINT.DAT file
    with open(file_path, "wb") as f:
        f.write(paint_data)


def extract_thumbnail(schematic_block: bytes) -> bytes:
    """
    Extracts the thumbnail data from a given schematic block.

    The schematic block is a chunk of data representing one schematic
    from the DESDOC.DAT file. This function isolates and returns the
    raw thumbnail data from within that block.

    Args:
        schematic_block: A bytes object representing a single schematic block.

    Returns:
        A bytes object containing the full thumbnail data (header and image).
    """
    # The absolute start of the first schematic is 0x148.
    # The absolute start of the first thumbnail is 0x200C.
    # The local offset is the difference: 0x200C - 0x148 = 0x1EC4.
    THUMBNAIL_LOCAL_OFFSET = 0x1EC4

    # The total size of the thumbnail data is 0x4010 bytes
    # (0x10 header + 0x4000 image data).
    THUMBNAIL_SIZE = 0x4010

    # Slice the block to get the thumbnail data
    thumbnail_data = schematic_block[THUMBNAIL_LOCAL_OFFSET:
                                     THUMBNAIL_LOCAL_OFFSET + THUMBNAIL_SIZE]

    return thumbnail_data


def bytes_to_image(thumbnail_bytes: bytes) -> Image.Image:
    """
    Converts raw ACFA thumbnail bytes into a Pillow Image object.

    Args:
        thumbnail_bytes: The 16400 bytes (0x4010) of thumbnail data.

    Returns:
        A Pillow Image object.
    """
    if len(thumbnail_bytes) != 0x4010:
        raise ValueError("Thumbnail data must be 0x4010 bytes long.")

    # The actual image data starts after the 16-byte header.
    dxt1_data = thumbnail_bytes[0x10:]

    # We need to construct a valid DDS header to make this readable by Pillow.
    # DDS header is 128 bytes.
    dds_header = bytearray(128)
    struct.pack_into('<4sI', dds_header, 0, b'DDS ', 124)  # Magic, Size
    # Flags
    struct.pack_into('<I', dds_header, 8, 0x1 | 0x2 | 0x4 | 0x1000 | 0x80000)
    struct.pack_into('<I', dds_header, 12, 128)  # Height
    struct.pack_into('<I', dds_header, 16, 256)  # Width
    struct.pack_into('<I', dds_header, 20, 16384)  # LinearSize
    # PixelFormat Sub-structure
    struct.pack_into('<I', dds_header, 76, 32)  # PixelFormat Size
    struct.pack_into('<I', dds_header, 80, 0x4)  # PixelFormat Flags (FourCC)
    struct.pack_into('<4s', dds_header, 84, b'DXT1')  # FourCC
    # Caps
    struct.pack_into('<I', dds_header, 108, 0x1000)  # DDSCAPS_TEXTURE

    dds_data = bytes(dds_header) + dxt1_data

    # Use an in-memory buffer to read the DDS data
    buffer = io.BytesIO(dds_data)
    image = Image.open(buffer)
    return image


def image_to_bytes(image: Image.Image) -> bytes:
    """
    Converts a Pillow Image object back into raw ACFA thumbnail bytes
    using the Wand/ImageMagick library for reliable compression.
    """
    if image.size != (256, 128):
        raise ValueError("Image must be 256x128 pixels.")

    # Convert the Pillow Image to a raw byte buffer (RGBA)
    img_rgba_bytes = image.convert('RGBA').tobytes()

    # Use a temporary file path for the DDS conversion
    with tempfile.NamedTemporaryFile(suffix='.dds', delete=False) as tmp_file:
        tmp_dds_path = tmp_file.name

    try:
        # Create a new Wand image and explicitly import the raw pixels.
        with WandImage(width=256, height=128) as img_wand:
            img_wand.import_pixels(data=img_rgba_bytes, channel_map='RGBA')

            # CRITICAL: Explicitly disable mipmap generation for the DDS file.
            img_wand.options['dds:mipmaps'] = '0'

            # Set the compression algorithm to DXT1 (BC1) and format to DDS
            img_wand.compression = 'dxt1'
            img_wand.format = 'dds'
            img_wand.save(filename=tmp_dds_path)

        # Read the temporary DDS file back into memory
        with open(tmp_dds_path, 'rb') as f:
            dds_data = f.read()

    finally:
        # Clean up the temporary file
        os.remove(tmp_dds_path)

    # Strip the 128-byte DDS header to get the raw DXT1 data
    DDS_HEADER_SIZE = 128
    dxt1_data = dds_data[DDS_HEADER_SIZE:]

    # Verify the size of the compressed data
    expected_size = 16384
    if len(dxt1_data) != expected_size:
        raise RuntimeError(
            f"DXT1 compression produced unexpected size: {len(dxt1_data)} bytes. Expected {expected_size}.")

    # Prepend the constant 16-byte ACFA header
    return ACFA_THUMBNAIL_HEADER + dxt1_data


def replace_thumbnail(schematic_block: bytes, new_thumbnail_bytes: bytes) -> bytes:
    """
    Replaces the thumbnail data within a schematic block.

    Args:
        schematic_block: The original schematic block bytes.
        new_thumbnail_bytes: The new thumbnail data (must be 0x4010 bytes).

    Returns:
        A new schematic block bytes object with the thumbnail replaced.
    """
    if len(new_thumbnail_bytes) != 0x4010:
        raise ValueError("New thumbnail data must be 0x4010 bytes long.")

    THUMBNAIL_LOCAL_OFFSET = 0x1EC4
    THUMBNAIL_SIZE = 0x4010

    # Create a mutable copy and replace the thumbnail data
    mutable_block = bytearray(schematic_block)
    mutable_block[THUMBNAIL_LOCAL_OFFSET:THUMBNAIL_LOCAL_OFFSET +
                  THUMBNAIL_SIZE] = new_thumbnail_bytes

    return bytes(mutable_block)


def save_schematic_block_as_ac4a(hex_block: bytes):
    """
    Saves a single schematic block to  'output/{schematic_name}_{designer_name}.ac4a' file.
    """
    sch_data = display_schematic_info(hex_block)
    schematic_name = sch_data['name']
    designer_name = sch_data['designer']
    output_path = f"output/{schematic_name}_{designer_name}.ac4a"

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "wb") as f:
        f.write(hex_block)


def load_schematic_block_from_ac4a(file_path: str) -> bytes:
    """
    Loads a schematic block from a .ac4a file.
    Returns the raw bytes representing the schematic block.
    """
    with open(file_path, "rb") as f:
        return f.read()
    

def insert_schematic(ac4a_path, desdoc_path):
    BLOCK_SIZE = 24280
    SCHEMATIC_COUNT_OFFSET = 5
    FIRST_MARKER_OFFSET = 0x148

    # Load files
    ac4a_data = load_file(ac4a_path)
    desdoc_data = bytearray(load_file(desdoc_path))

    # Extract current schematic count
    current_count = desdoc_data[SCHEMATIC_COUNT_OFFSET]
    print(f"Current active schematic count: {current_count}")

    # Calculate insertion offset
    insertion_offset = FIRST_MARKER_OFFSET + (current_count * BLOCK_SIZE)

    # Validate insertion space
    if insertion_offset + BLOCK_SIZE > len(desdoc_data):
        print("Error: Not enough space to insert schematic.")
        return

    # Insert schematic data
    desdoc_data[insertion_offset:insertion_offset +
                BLOCK_SIZE] = ac4a_data[:BLOCK_SIZE]

    # Update schematic count
    desdoc_data[SCHEMATIC_COUNT_OFFSET] += 1
    print(
        f"Inserted at slot {current_count + 1} (offset {hex(insertion_offset)}). New count: {desdoc_data[SCHEMATIC_COUNT_OFFSET]}")

    # Save modified DESDOC.DAT
    save_file(desdoc_path, desdoc_data)


def hex_dump(data: bytes, width: int = 16) -> str:
    lines = []
    for offset in range(0, len(data), width):
        chunk = data[offset:offset+width]
        hex_part = ' '.join(f'{b:02X}' for b in chunk)
        ascii_part = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        lines.append(f'{offset:08X}  {hex_part:<{width*3}}  {ascii_part}')
    return '\n'.join(lines)

dict_keys(['Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster', 'Back Booster', 'Side Booster', 'Overed Booster', 'Arm Unit', 'Back Unit', 'Shoulder Unit', 'Stabilizer Head Top', 'Stabilizer Head Side', 'Stabilizer Core Upper', 'Stabilizer Core Lower', 'Stabilizer Arms', 'Stabilizer Legs Back', 'Stabilizer Legs Upper', 'Stabilizer Legs Middle', 'Stabilizer Legs Lower'])


In [ ]:
loaded_block = load_schematic_block_from_ac4a(
    "output/Sbeu Tarakan_Vlabus.ac4a")

# Verify loaded block content by displaying its information
display_schematic_info(loaded_block)

In [ ]:
file_path = "sch_data/tab_sch_data/DESDOC.DAT"
blocks = extract_active_schematic_blocks(file_path)
print(f"Detected {len(blocks)} active schematic(s).")

for idx, block in enumerate(blocks, 1):
    print(f"Slot {idx}:")
    print(display_schematic_info(block))

In [ ]:
save_schematic_block_as_ac4a(blocks[6])

In [ ]:
insert_schematic("output/Sbeu Tarakan_Vlabus.ac4a",
                 "sch_data/tab_sch_data/DESDOC.DAT")

In [ ]:
def swap_part_in_ac4a_file(file_path, part_category, new_part_id):
    """
    Reads an .ac4a file, swaps a part, and saves it back to the same file.

    :param file_path: Path to the .ac4a schematic file.
    :param part_category: The category of the part to swap (e.g., 'Head', 'Core').
    :param new_part_id: The new part ID (as a string or int).
    """
    
    def _swap_part_in_block(block, part_category, new_part_id):
        # This is a helper function nested inside for self-containment.
        LOCAL_PARTS_OFFSET = 0xD8
        PART_ENTRY_SIZE = 2
        display_labels = [
            'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
            'Back Booster', 'Side Booster', 'Overed Booster',
            'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
            'Left Back Unit', 'Shoulder Unit'
        ]

        try:
            part_index = display_labels.index(part_category)
        except ValueError:
            raise ValueError(f"Invalid part category: {part_category}. Must be one of {display_labels}")

        offset = LOCAL_PARTS_OFFSET + part_index * PART_ENTRY_SIZE
        
        try:
            part_id_num = int(new_part_id)
        except ValueError:
            raise ValueError(f"Invalid part ID: {new_part_id}. Must be a number.")

        new_part_bytes = part_id_num.to_bytes(2, byteorder='big')

        mutable_block = bytearray(block)
        mutable_block[offset:offset + PART_ENTRY_SIZE] = new_part_bytes
        return bytes(mutable_block)

    # Main logic for the file-based swap
    print(f"--- Modifying {file_path} ---")
    
    # 1. Load the schematic block from the file
    try:
        original_block = load_schematic_block_from_ac4a(file_path)
        print("Original file loaded.")
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return

    # 2. Perform the swap in the loaded block
    modified_block = _swap_part_in_block(original_block, part_category, new_part_id)
    print(f"Swapped '{part_category}' to part ID {new_part_id}.")

    # 3. Save the modified block back to the original file
    save_file(file_path, modified_block)
    print(f"Successfully saved changes to {file_path}.")
    print("")


# --- Example Usage ---

# Specify the file to modify
ac4a_file_to_edit = "output/AfgankaXD_RAPTOR23512.ac4a"

# Check if the file exists before trying to modify it
if os.path.exists(ac4a_file_to_edit):
    # 1. Show the parts list *before* the change
    print("--- BEFORE ---")
    before_block = load_schematic_block_from_ac4a(ac4a_file_to_edit)
    before_info = display_schematic_info(before_block)
    # A more compact print for parts
    for part in before_info['parts']:
        print(f"{part['category']}: {part['part_id']} ({part['part_name']})")
    print("")

    # 2. Call the function to swap the 'Head' to part ID '1010'
    swap_part_in_ac4a_file(ac4a_file_to_edit, 'Right Back Unit', '2030')

    # 3. Show the parts list *after* the change to verify
    print("--- AFTER ---")
    after_block = load_schematic_block_from_ac4a(ac4a_file_to_edit)
    after_info = display_schematic_info(after_block)
    for part in after_info['parts']:
        print(f"{part['category']}: {part['part_id']} ({part['part_name']})")
        
    # Example of swapping it back
    # print("--- Swapping back for demonstration ---")
    # swap_part_in_ac4a_file(ac4a_file_to_edit, 'Head', '3010') # Assuming 3010 was the original

else:
    print(f"Example file not found: {ac4a_file_to_edit}")
    print("Please ensure the file exists, for example by running the cell that calls 'save_schematic_block_as_ac4a'.")

In [ ]:
import random
import os


def randomize_schematic_parts(file_path, part_mapping, new_name=None):
    """
    Reads an .ac4a file, randomizes its core parts, optionally renames it, 
    and saves it back to a new file, deleting the old one.
    Excludes debug parts (IDs starting with '9').

    :param file_path: Path to the .ac4a schematic file.
    :param part_mapping: The dictionary of all parts, loaded from the text file.
    :param new_name: An optional new name for the schematic.
    """
    # This function assumes that other necessary functions like
    # `load_schematic_block_from_ac4a`, `save_file`, and `display_schematic_info`
    # are already defined in the notebook.

    LOCAL_PARTS_OFFSET = 0xD8
    PART_ENTRY_SIZE = 2
    NAME_OFFSET = 1
    NAME_SIZE = 96  # 48 wchar_t = 96 bytes in UTF-16

    lookup_keys = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Arm Unit', 'Arm Unit', 'Back Unit', 'Back Unit', 'Shoulder Unit'
    ]
    display_labels = [
        'Head', 'Core', 'Arms', 'Legs', 'FCS', 'Generator', 'Main Booster',
        'Back Booster', 'Side Booster', 'Overed Booster',
        'Right Arm Unit', 'Left Arm Unit', 'Right Back Unit',
        'Left Back Unit', 'Shoulder Unit'
    ]

    try:
        original_block = load_schematic_block_from_ac4a(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return

    mutable_block = bytearray(original_block)
    print(f"--- Randomizing {file_path} ---")

    # Get original designer name for the new filename
    original_info = display_schematic_info(original_block)
    designer_name = original_info['designer']

    # Handle renaming if a new name is provided
    if new_name:
        if len(new_name) > (NAME_SIZE // 2) - 1:
            new_name = new_name[:(NAME_SIZE // 2) - 1]
            print(f"Warning: Name truncated to '{new_name}'")

        encoded_name = new_name.encode('utf-16-le')
        name_buffer = bytearray(NAME_SIZE)
        name_buffer[:len(encoded_name)] = encoded_name
        mutable_block[NAME_OFFSET:NAME_OFFSET + NAME_SIZE] = name_buffer
        print(f"  Renamed schematic to: {new_name}")

    for i, (lookup_key, display_label) in enumerate(zip(lookup_keys, display_labels)):
        part_id_list = [
            part_id for part_id in part_mapping.get(lookup_key, {}).keys()
            if not part_id.startswith('9')
        ]
        if not part_id_list:
            print(
                f"Warning: No valid parts found for category '{lookup_key}'. Skipping.")
            continue

        random_part_id_str = random.choice(part_id_list)
        random_part_id_num = int(random_part_id_str)
        offset = LOCAL_PARTS_OFFSET + i * PART_ENTRY_SIZE
        new_part_bytes = random_part_id_num.to_bytes(2, byteorder='big')
        mutable_block[offset:offset + PART_ENTRY_SIZE] = new_part_bytes
        part_name = part_mapping[lookup_key][random_part_id_str]
        print(f"  {display_label}: {random_part_id_str} ({part_name})")

    # Determine the output path
    if new_name:
        output_dir = os.path.dirname(file_path)
        new_filename = f"{new_name}.ac4a"
        output_path = os.path.join(output_dir, new_filename)
    else:
        output_path = file_path

    save_file(output_path, bytes(mutable_block))
    print(f"\nSuccessfully saved randomized schematic to {output_path}.")

    # If a new name was provided, delete the old file

In [ ]:
# --- Example Usage for Randomizer ---

# Specify the file to randomize and the new name
ac4a_file_to_randomize = "output/BBBB_Unknown.ac4a"
new_schematic_name = "Randomized AC"

# Check if the file exists
if os.path.exists(ac4a_file_to_randomize):
    # 1. Show the parts list *before* the change
    print("--- BEFORE RANDOMIZATION ---")
    before_block = load_schematic_block_from_ac4a(ac4a_file_to_randomize)
    before_info = display_schematic_info(before_block)
    # Get designer name for verification
    designer_name = before_info['designer']
    print(f"Name: {before_info['name']}")
    for part in before_info['parts']:
        print(f"  {part['category']}: {part['part_id']} ({part['part_name']})")
    print("")

    # 2. Call the randomizer function with a new name
    randomize_schematic_parts(ac4a_file_to_randomize,
                              part_mapping, new_name=new_schematic_name)
    print("")

    # 3. Construct the new file path and verify the changes
    new_file_path = os.path.join(
        "output", f"{new_schematic_name}.ac4a")

    print("--- AFTER RANDOMIZATION ---")
    if os.path.exists(new_file_path):
        after_block = load_schematic_block_from_ac4a(new_file_path)
        after_info = display_schematic_info(after_block)
        print(f"Name: {after_info['name']} (Saved to: {new_file_path})")
        for part in after_info['parts']:
            print(
                f"  {part['category']}: {part['part_id']} ({part['part_name']})")
    else:
        print(f"Error: New file not found at '{new_file_path}'")
else:
    print(f"Example file not found: {ac4a_file_to_randomize}")
    print("Please ensure the file exists, for example by running a cell that calls 'save_schematic_block_as_ac4a'.")

In [ ]:
from decal_utils import extract_decal_data, replace_decal_data, randomize_decal_data

# Load a schematic block
loaded_block = load_schematic_block_from_ac4a('output/Circus_Vlabus.ac4a')

# Extract original decal data and print its size
original_decal_data = extract_decal_data(loaded_block)
print(f"Extracted decal data size: {len(original_decal_data)} bytes")

# Generate random decal data
new_decal_data = randomize_decal_data()
print(f"Generated random decal data size: {len(new_decal_data)} bytes")

# Replace the decal data in the schematic block
modified_block = replace_decal_data(loaded_block, new_decal_data)

# Save the modified schematic block
save_schematic_block_as_ac4a(modified_block)
print("Successfully saved schematic with randomized decals.")

In [7]:
loaded_block = load_schematic_block_from_ac4a('output/Circus_Vlabus.ac4a')
thumbnail_data = extract_thumbnail(loaded_block)
print(f"Extracted thumbnail data size: {len(thumbnail_data)} bytes")
print(hex_dump(thumbnail_data))  # Print first 64 bytes of thumbnail data
image = bytes_to_image(thumbnail_data)
image.show()  # Display the image

Extracted thumbnail data size: 16400 bytes
00000000  10 00 00 80 00 00 00 00 00 00 00 00 00 00 40 00   ..............@.
00000010  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000020  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000030  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000040  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000050  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000060  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000070  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000080  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
00000090  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
000000A0  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
000000B0  FE FF FF F7 AA AA AA AA FE FF FF F7 AA AA AA AA   ................
000000C0  FE FF FF F7 AA AA AA AA

In [6]:
loaded_block = load_schematic_block_from_ac4a('output/Circus_Vlabus.ac4a')
new_image = Image.open("pepe.png")
new_thumbnail_bytes = image_to_bytes(new_image)
modified_block = replace_thumbnail(loaded_block, new_thumbnail_bytes)
save_schematic_block_as_ac4a(modified_block)

In [3]:
loaded_block = load_schematic_block_from_ac4a(
    "output/Circus_Vlabus.ac4a")

coloring_data, pattern_data, eye_color_data = extract_color_data(loaded_block)
print(f"Extracted coloring data size: {len(coloring_data)} bytes")
print(f"Extracted pattern data size: {len(pattern_data)} bytes")
print(f"Extracted eye color data size: {len(eye_color_data)} bytes")
print("Coloring Data:")
print(hex_dump(coloring_data[:64]))
print("Pattern Data:")
print(hex_dump(pattern_data))
print("Eye Color Data:")
print(hex_dump(eye_color_data))

new_coloring_data = randomize_colors(coloring_data)

print("New Coloring Data:")
print(hex_dump(new_coloring_data[:64]))
modified_block = replace_color_data(
    loaded_block,
    new_colors=new_coloring_data,
)
save_schematic_block_as_ac4a(modified_block)

Extracted coloring data size: 816 bytes
Extracted pattern data size: 36 bytes
Extracted eye color data size: 4 bytes
Coloring Data:
00000000  6E 72 76 FF 35 34 33 FF 2E 30 32 FF 24 26 28 FF   nrv.543..02.$&(.
00000010  6E 6C 68 FF 24 26 28 FF 6E 72 76 FF 35 34 33 FF   nlh.$&(.nrv.543.
00000020  2E 30 32 FF 24 26 28 FF 6E 6C 68 FF 24 26 28 FF   .02.$&(.nlh.$&(.
00000030  6E 72 76 FF 35 34 33 FF 2E 30 32 FF 24 26 28 FF   nrv.543..02.$&(.
Pattern Data:
00000000  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000010  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000020  00 00 00 00                                       ....
Eye Color Data:
00000000  FF 19 19 FF                                       ....
New Coloring Data:
00000000  35 9D C2 FF 49 C1 8E FF 72 C7 71 FF 6B 18 9D FF   5...I...r.q.k...
00000010  AD 8B 9B FF 43 A2 E1 FF F0 0F 1C FF 46 8C 14 FF   ....C.......F...
00000020  AF CD 6C FF 55 F0 71 FF 19 A9 77 FF 4A 04 6D FF   ..l.U.q...w.J

In [6]:
loaded_block = load_schematic_block_from_ac4a(
    "output/Bomber_Inveigh.ac4a")

decal_data = extract_decal_data(loaded_block)
print(f"Extracted decal data size: {len(decal_data)} bytes")
print("Decal Data:")
print(hex_dump(decal_data[:64]))

random_decal_data = randomize_decal_data()
print("Random Decal Data:")
print(hex_dump(random_decal_data[:64]))
modified_block = replace_decal_data(loaded_block, random_decal_data)
save_schematic_block_as_ac4a(modified_block)


Extracted decal data size: 6560 bytes
Decal Data:
00000000  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000010  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000020  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000030  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
Random Decal Data:
00000000  8E BC 0E CF 70 A7 EA FE 28 E8 E8 A5 84 26 86 24   ....p...(....&.$
00000010  1E A2 5E 57 51 2C B9 EE 99 8A F4 8C 36 AF F4 3D   ..^WQ,......6..=
00000020  FB 2C 84 44 35 66 6B DE F0 E1 80 E9 10 A5 2E 94   .,.D5fk.........
00000030  78 DB D0 4E B8 15 F8 88 3B 32 10 23 B0 BC 6D C8   x..N....;2.#..m.


In [15]:
loaded_paint_data = parse_paint_dat("paint_data/PAINT.DAT")
print(f"Extracted {len(loaded_paint_data)} emblem(s) from PAINT.DAT.")
print("Sixth Emblem Data:")
print(hex_dump(loaded_paint_data[5]))
print(parse_emblem_data(loaded_paint_data[5]))

for i in range(16):
    random_emblem = generate_random_emblem(num_layers=i+1)
    append_emblem_to_paint_dat("paint_data/PAINT.DAT", random_emblem)

Extracted 8 emblem(s) from PAINT.DAT.
Sixth Emblem Data:
00000000  02 00 00 00 00 01 07 20 20 00 00 00 B2 7D 00 20   .......  ....}. 
00000010  20 00 00 08 00 00 00 00 00 00 00 00 00 00 00 00    ...............
00000020  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000030  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000040  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000050  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000060  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000070  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
00000080  00 00 00 00                                       ....
{'type': 2, 'unknown_header': '000000', 'layers': [{'layer_index': 0, 'angle': 0, 'image_id': 1, 'color': 7, 'width': 32, 'height': 32, 'x_position': 0, 'y_position': 0, 'flags': {'raw_byte': '0x00', 'negative_angle': False, 'negative_x': False, 'negative_y': Fa